## Strategy Statement Generator and Optimizer

This notebook implements an iterative approach to generating and refining business strategy statements based on user input.

### Components:
1. User Input Collection
2. Strategy Statement Generation
3. Evaluation and Refinement Loop

In [1]:
import os
import sys
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
project_dir = os.path.dirname(os.path.dirname(notebook_dir))
sys.path.insert(0, project_dir)

from Jupyter.util import llm_call, extract_xml

from dataclasses import dataclass
from typing import List, Dict, Optional

@dataclass
class BusinessContext:
    industry: str
    target_market: str
    unique_value: str

@dataclass
class StrategyStatements:
    vision: str
    mission: str
    objectives: List[str]

def collect_user_input() -> BusinessContext:
    """Collect the three key pieces of business context from user."""
    print("Please answer these three questions about your business:")
    industry = input("1. What industry are you in? ")
    market = input("2. Who is your target market? ")
    value = input("3. What unique value do you provide? ")
    
    return BusinessContext(industry, market, value)

def generate_strategy(prompt: str, context: BusinessContext, feedback: str = "") -> tuple[str, StrategyStatements]:
    """Generate strategy statements based on business context and any previous feedback."""
    full_prompt = f"{prompt}\nContext:\n"
    full_prompt += f"Industry: {context.industry}\n"
    full_prompt += f"Target Market: {context.target_market}\n"
    full_prompt += f"Unique Value: {context.unique_value}\n"
    if feedback:
        full_prompt += f"\nPrevious Feedback: {feedback}"

    response = llm_call(full_prompt)
    thoughts = extract_xml(response, "thoughts")
    statements = extract_xml(response, "statements")
    
    # Parse statements into structured format
    vision = extract_xml(statements, "vision")
    mission = extract_xml(statements, "mission")
    objectives = extract_xml(statements, "objectives").split('\n')
    
    return thoughts, StrategyStatements(vision, mission, objectives)

def evaluate_strategy(prompt: str, statements: StrategyStatements) -> tuple[str, str]:
    """Evaluate strategy statements against best practices."""
    full_prompt = f"{prompt}\n"
    full_prompt += f"Vision: {statements.vision}\n"
    full_prompt += f"Mission: {statements.mission}\n"
    full_prompt += f"Objectives:\n{chr(10).join(statements.objectives)}"
    
    response = llm_call(full_prompt)
    evaluation = extract_xml(response, "evaluation")
    feedback = extract_xml(response, "feedback")
    
    return evaluation, feedback

In [2]:
generator_prompt = """
Generate compelling strategy statements based on the business context provided.

Guidelines:
- Vision: Should be aspirational, future-focused, and memorable
- Mission: Should be clear, actionable, and focused on current purpose
- Objectives: Should be SMART (Specific, Measurable, Achievable, Relevant, Time-bound)

Format your response as:
<thoughts>Your reasoning about the strategy</thoughts>
<statements>
    <vision>The vision statement</vision>
    <mission>The mission statement</mission>
    <objectives>
    1. First objective
    2. Second objective
    3. Third objective
    </objectives>
</statements>
"""

evaluator_prompt = """
Evaluate these strategy statements against best practices:

Vision Criteria:
- Aspirational but achievable
- Clear and memorable
- Future-focused

Mission Criteria:
- States clear purpose
- Actionable
- Aligned with vision

Objectives Criteria:
- SMART format
- Aligned with mission
- Balanced and comprehensive

Format your response as:
<evaluation>PASS, NEEDS_IMPROVEMENT, or FAIL</evaluation>
<feedback>Specific suggestions for improvement</feedback>
"""

# Test the workflow
context = collect_user_input()
thoughts, statements = generate_strategy(generator_prompt, context)
evaluation, feedback = evaluate_strategy(evaluator_prompt, statements)

print("\nGenerated Strategy:")
print(f"Vision: {statements.vision}")
print(f"Mission: {statements.mission}")
print("Objectives:")
for obj in statements.objectives:
    print(f"- {obj}")

print(f"\nEvaluation: {evaluation}")
print(f"Feedback: {feedback}")

Please answer these three questions about your business:

Generated Strategy:
Vision: To reinvent how consumers experience energy - making it simpler, more affordable, and unified.
Mission: We broker energy solutions that allow utilities to focus on delivery while giving consumers the best prices and the convenience of a single bill for all their energy needs.
Objectives:
- 1. Expand to serve 250,000 energy customers across 10 states by 2025
- 2. Achieve and maintain a customer Net Promoter Score of 60+ to reflect high satisfaction 
- 3. Grow revenue at a 30% CAGR over the next 3 years to $50M by diversifying energy offerings

Evaluation: PASS
Feedback: The vision statement is aspirational, describing a future state of reinventing the consumer energy experience. It's clear and memorable, focusing on simplicity, affordability, and unification. No significant improvements needed.
